# Adaptar Tablas

Este Notebook sirve para crear un Excel con el formato adecuado para hacer clustering y estudiar los resultados

## Importar Librerias y Visualizar Tabla

In [12]:
import numpy as np
import pandas as pd
import pdb

name = 'Jobs.xlsx'
d_input = pd.read_excel(name, keep_default_na=False, na_values='..')

d_input.head()

,Country Name,Country Code,Series Name,Series Code,Most Recent Value [YR20111]
0,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,43.000000
1,Afghanistan,AFG,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,76.733600
2,Afghanistan,AFG,Age dependency ratio (% of working-age populat...,SP.POP.DPND,89.773777
3,Afghanistan,AFG,Agriculture value added per worker (constant 2...,EA.PRD.AGRI.KD,395.782545
4,Afghanistan,AFG,"Agriculture, value added (% of GDP)",NV.AGR.TOTL.ZS,23.966824


## Eliminar columnas innecesarias

In [13]:
todel_columns = ['Series Code']
for column in todel_columns:
    d_input.__delitem__(column)
d_input.head()

,Country Name,Country Code,Series Name,Most Recent Value [YR20111]
0,Afghanistan,AFG,Access to electricity (% of population),43.000000
1,Afghanistan,AFG,"Adolescent fertility rate (births per 1,000 wo...",76.733600
2,Afghanistan,AFG,Age dependency ratio (% of working-age populat...,89.773777
3,Afghanistan,AFG,Agriculture value added per worker (constant 2...,395.782545
4,Afghanistan,AFG,"Agriculture, value added (% of GDP)",23.966824


## Dejar paises como única fila

In [14]:
#El metodo np.unique ordena las listas 
#es necesario devolver la lista a su orden original
#===================================================
'''
firstcolumns = ['Country Name','Series Name','Country Code']
newcolumns = [] #column labels with data
for fcolumn in firstcolumns:
    #print fcolumn
    #np_column = np.array(d_input[fcolumn])     #castear a numpy array
    #indexes_d = np.unique(np_column, return_index=True)[1]   #tomar filas sin repeticion
    #np_column = [np_column[index] for index in sorted(indexes_d)]   #reordenar segun el orden original
    #tupla = (fcolumn,newcolumn)
    #pdb.set_trace()
    #newcolumns.append(tupla)
    if fcolumn == 'Series Name':
        features_d = np_column    #Lista con las caracteristicas
'''

#===================================================
paises_d = np.array(d_input['Country Name'])
indexes_d = np.unique(paises_d, return_index=True)[1]
paises_d = [paises_d[index] for index in sorted(indexes_d)]

features_d = np.array(d_input['Series Name'])
indexes_d = np.unique(features_d, return_index=True)[1]
features_d = [features_d[index] for index in sorted(indexes_d)]

codes_d = np.array(d_input['Country Code'])
indexes_d = np.unique(codes_d, return_index=True)[1]
codes_d = [codes_d[index] for index in sorted(indexes_d)]

#====================================================

columns_d = [('Country Name',paises_d),('Country Code',codes_d)]

empty = 0
notempty = 0
for c_feature in features_d:
    vals = d_input[d_input['Series Name'] == c_feature]['Most Recent Value [YR20111]']   #Valores de una caracteristica para todos los paises
    xvals = vals.tolist()
    if (~np.isnan(np.array(xvals))).any():      #No considerar columnas con puros NaN
        columns_d.append((c_feature,xvals))
        notempty+=1
    else:
        empty+=1

#pdb.set_trace()
new_input = pd.DataFrame.from_items(columns_d)
new_input.head()


,Country Name,Country Code,Access to electricity (% of population),"Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),Agriculture value added per worker (constant 2005 US$),"Agriculture, value added (% of GDP)","Agriculture, value added (annual % growth)","Agriculture, value added (constant 2005 US$)","Air transport, registered carrier departures worldwide",...,Union members (% of total paid employees),Urban population,Urban population (% of total),Vehicles (per km of road),"Vulnerable employment, female (% of female employment)","Vulnerable employment, male (% of male employment)","Wage and salaried workers, female (% of females employed)","Wage and salaried workers, total (% of total employed)","Wage and salary workers, male (% of males employed)",Weeks of severance pay for redundancy of experienced workers
0,Afghanistan,AFG,43.00000,76.7336,89.773777,395.782545,23.966824,1.90,2.689738e+09,25389.376787,...,NaN,8312341.0,26.282,41.579259,NaN,NaN,NaN,NaN,NaN,26.000000
1,Albania,ALB,100.00000,21.4896,44.912136,3914.815249,22.671257,1.98,2.317571e+09,1994.750856,...,NaN,1632744.0,56.409,12.454833,62.0,54.799999,36.900002,40.200001,42.900002,42.857143
2,Algeria,DZA,100.00000,10.7914,51.536631,3718.553866,10.011399,10.30,1.185475e+10,59588.200311,...,NaN,27304259.0,70.129,35.604091,24.4,27.500000,73.699997,69.000000,67.900002,13.000000
3,American Samoa,ASM,59.32891,NaN,NaN,NaN,NaN,NaN,NaN,9138.074929,...,NaN,48374.0,87.264,NaN,NaN,NaN,98.500000,96.900002,97.000000,NaN
4,Andorra,AND,100.00000,NaN,NaN,4149.463596,0.523888,NaN,1.244839e+07,NaN,...,NaN,62329.0,85.633,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Quitar columnas con muchos NaNs

In [15]:
coef = 0.2   # nans/total debe dar superior a este numero
deleted = 0
output = new_input
todel = []
itodel = []

for k in range(2,len(new_input.keys())):    
    llave = new_input.keys()[k]
    valores = new_input[llave] 
    total = valores.shape[0]
    nans = np.count_nonzero(np.isnan(valores))
    xnans = 0    
    
    if coef < float(nans)/float(total):
        todel.append(llave)
        itodel.append(k)
        deleted += 1

for column in todel:
    output.__delitem__(column)


print("Se eliminaron "+str(deleted)+" columnas")


deleted = 0
new_output = output.T
todel = []
itodel = []

for k in range(len(new_output.keys())):    
    llave = new_output.keys()[k]
    valores = new_output[llave] #
    xvalores = np.array(valores)[2:-1]
    xvalores = xvalores.astype(float)
    total = xvalores.shape[0]
    #pdb.set_trace()
    nans = np.count_nonzero(np.isnan(xvalores))  
    
    if coef < float(nans)/float(total):
        todel.append(llave)
        itodel.append(k)
        deleted += 1

for column in todel:
    new_output.__delitem__(column)


print("Se eliminaron "+str(deleted)+" filas")

out = new_output.T
out.head()

Se eliminaron 89 columnas
Se eliminaron 35 filas


,Country Name,Country Code,Access to electricity (% of population),"Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),Agriculture value added per worker (constant 2005 US$),"Agriculture, value added (% of GDP)","Agriculture, value added (annual % growth)","Agriculture, value added (constant 2005 US$)","Air transport, registered carrier departures worldwide",...,"Total employment, total (ages 15+)","Unemployment, female (% of female labor force) (modeled ILO estimate)","Unemployment, male (% of male labor force) (modeled ILO estimate)","Unemployment, total (% of total labor force) (modeled ILO estimate)","Unemployment, youth female (% of female labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)",Urban population,Urban population (% of total),Weeks of severance pay for redundancy of experienced workers
0,Afghanistan,AFG,43,76.7336,89.7738,395.783,23.9668,1.9,2.68974e+09,25389.4,...,7.33265e+06,12.2,7.1,8,22.8,16.4,17.5,8.31234e+06,26.282,26
1,Albania,ALB,100,21.4896,44.9121,3914.82,22.6713,1.98,2.31757e+09,1994.75,...,1.08147e+06,13.8,17.6,16,24.1,32,28.7,1.63274e+06,56.409,42.8571
2,Algeria,DZA,100,10.7914,51.5366,3718.55,10.0114,10.3,1.18547e+10,59588.2,...,1.09043e+07,16.8,8.4,9.8,38.7,21,24,2.73043e+07,70.129,13
5,Angola,AGO,37,166.603,100.638,866.829,9.43631,1.24901,5.64132e+09,13817.6,...,7.94796e+06,7.2,6.5,6.8,11.1,10.1,10.6,1.04842e+07,43.274,54.1667
6,Antigua and Barbuda,ATG,90.8754,45.6642,46.3485,2311.76,2.24348,0.596511,1.84941e+07,46407.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21989,24.19,48


## Exportar

In [16]:
writer = pd.ExcelWriter('Jobs_proc.xlsx')
out.to_excel(writer,'Data')
writer.save()